In [12]:
import numpy as np
import json
import plotly.graph_objects as go


METADATA = {'Contributor': 'T. Dunn'}
FSIZE = [5,4]
FSIZEWIDE = [10,4]
FDPI = 100
SAVEFIGS = True
SAVEDATA = False

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

ENCOUNTER_DIFFICULTIES = {
    'Easy':   {
        'XP': [25,50,75,125,250,300,350,450,550,600,800,1000,1100,1250,1400,1600,2000,2100,2400,2800], 
        'color': '#1F77B4'
    },
    'Medium': {
        'XP': [50,100,150,250,500,600,750,900,1100,1200,1600,2000,2200,2500,2800,3200,3900,4200,4900,5700], 
        'color': '#FF7F0E'
    },
    'Hard':   {
        'XP': [75,150,225,375,750,900,1100,1400,1600,1900,2400,3000,3400,3800,4300,4800,5900,6300,7300,8500], 
        'color': '#2CA02C'
    },
    'Deadly': {
        'XP': [100,200,400,500,1100,1400,1700,2100,2400,2800,3600,4500,5100,5700,6400,7200,8800,9500,10900,12700], 
        'color': '#D62728'
    },
    'Daily':  {
        'XP': [300,600,1200,1700,3500,4000,5000,6000,7500,9000,10500,11500,13500,15000,18000,20000,25000,27000,30000,40000], 
        'color': '#9467BD'
    },
}

In [10]:
# Setup template and html config for plotly figures.

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            autorange=False,
            automargin=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            autorange=False,
            automargin=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        legend=dict(
            #bordercolor='Black',
            #borderwidth=1,
            font_size=10,
            #tracegroupgap=5,
            #orientation='h',
        )
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': False, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}


In [50]:
# Plots average encounter XP for PCs, calculated for a full adventuring day with two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure()

enc_XP_budget = 0.5*np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
for diff in ['Easy','Medium','Hard','Deadly']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    pc_data_file = f'./pc data - {diff} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    enc_XP = np.zeros(20)
    for pc_class in PC_CLASSES:
        enc_XP += np.array(pc_data[pc_class]['encounter XP mean'])/len(PC_CLASSES)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=enc_XP/enc_XP_budget,
        mode='lines+markers', 
        line=dict(color=d['color'], dash='solid'),
        marker={'size': 10},
        unselected=dict(marker=dict(opacity=0.0)),
        name=diff + ' encounters',
        hovertemplate=
            '<b>' + diff + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'XP ratio %{y:.2f}'+
            '<extra></extra>'
    ))

ref_XP = 0.5*np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
fig.add_trace(go.Scatter(
    x=levels, 
    y=ref_XP/enc_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    name='half Daily XP threshold',
    showlegend=False,
    hoverinfo='skip'
))

ref_XP = np.array(ENCOUNTER_DIFFICULTIES['Deadly']['XP'])
fig.add_trace(go.Scatter(
    x=levels, 
    y=ref_XP/enc_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dashdot'),
    name='Deadly XP threshold',
    hoverinfo='skip'
))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=600,
    #height=800,
    xaxis=dict(
        title_text='level',
        range=[0.8, 20.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='calculated encounter XP / half Daily XP',
        range=[0, 1.5],
        tick0=0, dtick=0.2,
        minor=dict(tick0=0, dtick=0.1),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig.write_html(
        f'./fig-half-daily-xp-vs-level-adventuring-days.html', 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )

In [49]:
# Plots average daily XP for PCs, calculated for a full adventuring day with two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure()

enc_XP_budget = np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
for diff in ['Easy','Medium','Hard','Deadly']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    pc_data_file = f'./pc data - {diff} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    enc_XP = np.zeros(20)
    for pc_class in PC_CLASSES:
        enc_XP += np.array(pc_data[pc_class]['daily XP mean'])/len(PC_CLASSES)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=enc_XP/enc_XP_budget,
        mode='lines+markers', 
        line=dict(color=d['color'], dash='solid'),
        marker={'size': 10},
        unselected=dict(marker=dict(opacity=0.0)),
        name=diff + ' encounters',
        hovertemplate=
            '<b>' + diff + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'XP ratio %{y:.2f}'+
            '<extra></extra>'
    ))

ref_XP = np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
fig.add_trace(go.Scatter(
    x=levels, 
    y=ref_XP/enc_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    name='Daily XP budget',
    showlegend=False,
    hoverinfo='skip'
))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=600,
    #height=800,
    xaxis=dict(
        title_text='level',
        range=[0.8, 20.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='calculated daily XP / Daily XP',
        range=[0, 1.6],
        tick0=0, dtick=0.2,
        minor=dict(tick0=0, dtick=0.1),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
        orientation='v',
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig.write_html(
        f'./fig-full-daily-xp-vs-level-adventuring-days.html', 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )

In [64]:
# Plots average daily effective DPR for PCs, calculated for a full adventuring day with two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure()

for diff in ['Easy','Medium','Hard','Deadly']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    pc_data_file = f'./pc data - {diff} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    eDPR = np.zeros(20)
    for pc_class in PC_CLASSES:
        eDPR += np.array(pc_data[pc_class]['effective damage per round mean'])/len(PC_CLASSES)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eDPR,
        mode='lines+markers', 
        line=dict(color=d['color'], dash='solid'),
        marker={'size': 10},
        unselected=dict(marker=dict(opacity=0.0)),
        name=diff + ' encounters',
        hovertemplate=
            '<b>' + diff + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'eDPR %{y:.1f}'+
            '<extra></extra>'
    ))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=600,
    #height=800,
    xaxis=dict(
        title_text='level',
        range=[0.8, 20.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='effective damage per round',
        range=[0, 100],
        tick0=0, dtick=10.0,
        minor=dict(tick0=0, dtick=5.0),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
        orientation='v',
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig.write_html(
        f'./fig-full-daily-edpr-vs-level-adventuring-days.html', 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )

In [67]:
# Plots average daily effective DPR for PCs, calculated for a full adventuring day with two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure()

for diff in ['Easy','Medium','Hard','Deadly']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    pc_data_file = f'./pc data - {diff} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    eDPR = np.zeros(20)
    for pc_class in PC_CLASSES:
        eDPR += np.array(pc_data[pc_class]['effective hit points mean'])/len(PC_CLASSES)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eDPR,
        mode='lines+markers', 
        line=dict(color=d['color'], dash='solid'),
        marker={'size': 10},
        unselected=dict(marker=dict(opacity=0.0)),
        name=diff + ' encounters',
        hovertemplate=
            '<b>' + diff + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'eDPR %{y:.2f}'+
            '<extra></extra>'
    ))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=600,
    #height=800,
    xaxis=dict(
        title_text='level',
        range=[0.8, 20.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='effective hit points',
        range=[0, 1000],
        tick0=0, dtick=200.0,
        minor=dict(tick0=0, dtick=100.0),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
        orientation='v',
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig.write_html(
        f'./fig-full-daily-edpr-vs-level-adventuring-days.html', 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )

In [48]:
# Plots average daily XP for PCs relative to Medium adventuring day, calculated for a full adventuring day with two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure()

enc_XP_budget = np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
diffs = {
    'Easy': {
        'daily XP': np.zeros(20)
    },
    'Medium': {
        'daily XP': np.zeros(20)
    },
    'Hard': {
        'daily XP': np.zeros(20)
    },
    'Deadly': {
        'daily XP': np.zeros(20)
    }
}
for d in diffs:
    diff = diffs[d]
    ediff = ENCOUNTER_DIFFICULTIES[d]
    pc_data_file = f'./pc data - {d} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    enc_XP = np.zeros(20)
    for pc_class in PC_CLASSES:
        enc_XP += np.array(pc_data[pc_class]['daily XP mean'])
    enc_XP /= len(PC_CLASSES)
    diff['daily XP'] = enc_XP/enc_XP_budget

    """fig.add_trace(go.Scatter(
        x=levels, 
        y=enc_XP/enc_XP_budget,
        mode='lines+markers', 
        line=dict(color=ediff['color'], dash='solid'),
        marker={'size': 10},
        unselected=dict(marker=dict(opacity=0.0)),
        name=d + ' encounters',
        hovertemplate=
            '<b>' + d + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'XP ratio %{y:.2f}'+
            '<extra></extra>'
    ))"""

for d in diffs:
    y = diffs[d]['daily XP']/diffs['Medium']['daily XP']
    fig.add_trace(go.Scatter(
        x=levels, 
        y=y,
        mode='markers+lines', 
        #line=dict(color='black', dash='dash'),
        name=d+' encounters',
        showlegend=True,
        hovertemplate=
            '<b>' + d + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'XP ratio %{y:.2f}'+
            '<extra></extra>'
    ))
    print(d, y.mean(), np.median(y))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=600,
    #height=800,
    xaxis=dict(
        title_text='level',
        range=[0.8, 20.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='daily XP / Medium daily XP',
        range=[0.8, 1.3],
        tick0=0, dtick=0.1,
        minor=dict(tick0=0, dtick=0.05),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
        orientation='v',
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig.write_html(
        f'./fig-daily-xp-ratio-vs-level.html', 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )

Easy 0.9216549983630635 0.9160972410676849
Medium 1.0 1.0
Hard 1.0555959608737715 1.0612941051283817
Deadly 1.1196091719372814 1.1238302291380462


In [16]:
# plots the ratio between the Deadly and Medium daily XPs for each classes.
from datetime import date
import plotly.express as px
from bs4 import BeautifulSoup

enc_diffs = ['Easy','Medium','Hard','Deadly']
levels = np.array(range(1,21,1))

fig = go.Figure()

groups = {
    'martial': {
        'classes': [], 
        'xp mean': np.zeros(len(levels))
    },
    'martial-spellcaster': {
        'classes': [], 
        'xp mean': np.zeros(len(levels))
    },
    'spellcaster': {
        'classes': [], 
        'xp mean': np.zeros(len(levels))
    },
}
xp_mean = np.zeros(len(levels))
for pc in PC_CLASSES:
    pc_class = PC_CLASSES[pc]
    pc_xp = np.zeros([20, len(enc_diffs)])
    for i in range(len(enc_diffs)):
        diff = enc_diffs[i]
        with open(f'./pc data - {diff} 2 short rests.json', 'r') as fin:
            pc_data = json.load(fin)
        
        pc_xp[:,i] = np.array(pc_data[pc]['daily XP mean'])

    for i in range(20):
        pc_xp[i,:] /= pc_xp[i,1]

    fig.add_trace(go.Scatter(
        x=levels, 
        y=pc_xp[:,3],
        mode='lines+markers', 
        line=dict(color=pc_class['color'], dash='solid'),
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        #legendgroup=pc_class['group'],
        #legendgrouptitle_text=pc_class['group'],
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'level %{x}<br>'+
            'XP delta %{y:.2f}'+
            '<extra></extra>'
    ))
    
    groups[pc_class['group']]['classes'] += [pc]
    groups[pc_class['group']]['xp mean'] += pc_xp[:,3]

for g in groups:
    group = groups[g]

    group['xp mean'] /= len(group['classes'])

    fig.add_trace(go.Scatter(
        x=levels, 
        y=group['xp mean'],
        mode='lines', 
        line=dict(dash='dash'),
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=g,
        #legendgroup=g,
        #legendgrouptitle_text=g,
        #hoverinfo='skip',
        hovertemplate=
            '<b>'+g+'</b><br>'+
            'level %{x}<br>'+
            'XP delta %{y:.2f}'+
            '<extra></extra>'
    ))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=600,
    #height=800,
    xaxis=dict(
        title_text='level',
        range=[-0.2, 20.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='Deadly / Medium encounter XP',
        range=[0.90, 1.4],
        tick0=0, dtick=0.1,
        minor=dict(tick0=0, dtick=0.05),
    ),
    legend=dict(
        #groupclick='toggleitem',
        #itemsizing= 'constant',
        #yanchor='top',  y=0.99,
        #xanchor='left', x=0.01,
        #orientation='v',
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig_file = f'./fig-class-daily-xp-ratio-vs-level.html'
    fig.write_html(
        fig_file, 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )

    # add style to figure
    with open(fig_file, 'r') as fin:
        fig_soup = BeautifulSoup(fin, 'html.parser')

    fig_soup.div['style'] = 'width:700px; min-width:50%; max-width:100%'
    with open(fig_file, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [68]:
# plots the ratio between the Deadly and Medium encounter XPs for each class
enc_diffs = ['Easy','Medium','Hard','Deadly']
levels = np.array(range(1,21,1))

fig = go.Figure()

xp_mean = np.zeros(len(levels))
for pc in PC_CLASSES:
    pc_xp = np.zeros([20, len(enc_diffs)])
    for i in range(len(enc_diffs)):
        diff = enc_diffs[i]
        with open(f'./pc data - {diff} 2 short rests.json', 'r') as fin:
            pc_data = json.load(fin)
        
        pc_xp[:,i] = np.array(pc_data[pc]['effective hit points mean'])

    for i in range(20):
        pc_xp[i,:] /= pc_xp[i,1]
    
    fig.add_trace(go.Scatter(
        x=levels, 
        y=pc_xp[:,3],
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[pc]['color'], dash='solid'),
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'level %{x}<br>'+
            'XP delta %{y:.2f}'+
            '<extra></extra>'
    ))
    
    xp_mean += pc_xp[:,3]

xp_mean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=xp_mean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=600,
    #height=800,
    xaxis=dict(
        title_text='level',
        range=[-0.2, 20.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='Deadly / Medium effective hit points',
        range=[0.90, 1.4],
        tick0=0, dtick=0.1,
        minor=dict(tick0=0, dtick=0.05),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig.write_html(
        f'./fig-class-daily-xp-delta-vs-encounter-difficulty.html', 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )

In [69]:
# plots the ratio between the Deadly and Medium encounter XPs for each class
enc_diffs = ['Easy','Medium','Hard','Deadly']
levels = np.array(range(1,21,1))

fig = go.Figure()

xp_mean = np.zeros(len(levels))
for pc in PC_CLASSES:
    pc_xp = np.zeros([20, len(enc_diffs)])
    for i in range(len(enc_diffs)):
        diff = enc_diffs[i]
        with open(f'./pc data - {diff} 2 short rests.json', 'r') as fin:
            pc_data = json.load(fin)
        
        pc_xp[:,i] = np.array(pc_data[pc]['effective damage per round mean'])

    for i in range(20):
        pc_xp[i,:] /= pc_xp[i,1]
    
    fig.add_trace(go.Scatter(
        x=levels, 
        y=pc_xp[:,3],
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[pc]['color'], dash='solid'),
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'level %{x}<br>'+
            'XP delta %{y:.2f}'+
            '<extra></extra>'
    ))
    
    xp_mean += pc_xp[:,3]

xp_mean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=xp_mean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=600,
    #height=800,
    xaxis=dict(
        title_text='level',
        range=[-0.2, 20.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='Deadly / Medium effective DPR',
        range=[0.90, 1.5],
        tick0=0, dtick=0.1,
        minor=dict(tick0=0, dtick=0.05),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig.write_html(
        f'./fig-class-daily-xp-delta-vs-encounter-difficulty.html', 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )